In [ ]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install -U accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 3.2 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

In [ ]:
raw_datasets = load_dataset("yyu/amazon-attrprompt")

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['_id', 'text'],
        num_rows: 13820
    })
    validation: Dataset({
        features: ['_id', 'text'],
        num_rows: 230
    })
    test: Dataset({
        features: ['_id', 'text'],
        num_rows: 1130
    })
})

In [ ]:
raw_datasets["validation"][100]

{'_id': 10,
 'text': "My first Aquis mitt was rectangular. I like this one better. Rounding the edges makes sense since it is meant to be worn on the hand. Aquis microfiber is the only fabric I use on my face. It is gentle on the skin, yet cleans more thoroughly than anything else I have used. (I find it amazing that both attributes apply to the same fabric!) If the manufacturer is listening, I would like to be able to buy the mitts in sets of 2. This would be perfect for bathing baby children. There is a slit in the seam, on the thumb side, so your thumb can protrude. You can grip lift without removing your mitts. Some size choices (S M L XL) would be nice, as this mitt would not fit a large man's han"}

In [ ]:
# checkpoint = "bert-base-cased" # 109M params
# checkpoint = "roberta-base" # 125M params
# checkpoint = "roberta-base-squad2" # 124M params
# checkpoint = "tinyroberta-squad2" # 81.5M params
# checkpoint = "albert-base-v2" # 11.8M params
# checkpoint = "albert-large-v2" # 17.9M params
# checkpoint = "albert-xlarge-v2" # 58M params
# checkpoint = "albert-xxlarge-v2" # 223M params
# checkpoint = "distilbert-base-uncased-finetuned-sst-2-english" # 67M params
checkpoint = "distilbert-base-cased" # 65.8M params

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["text"], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, )

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Map:   0%|          | 0/13820 [00:00<?, ? examples/s]

Map:   0%|          | 0/230 [00:00<?, ? examples/s]

Map:   0%|          | 0/1130 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['_id', 'text', 'input_ids', 'attention_mask'],
        num_rows: 13820
    })
    validation: Dataset({
        features: ['_id', 'text', 'input_ids', 'attention_mask'],
        num_rows: 230
    })
    test: Dataset({
        features: ['_id', 'text', 'input_ids', 'attention_mask'],
        num_rows: 1130
    })
})

In [ ]:
tokenized_datasets = tokenized_datasets.rename_column("_id", "labels")

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['labels', 'text', 'input_ids', 'attention_mask'],
        num_rows: 13820
    })
    validation: Dataset({
        features: ['labels', 'text', 'input_ids', 'attention_mask'],
        num_rows: 230
    })
    test: Dataset({
        features: ['labels', 'text', 'input_ids', 'attention_mask'],
        num_rows: 1130
    })
})

In [ ]:
set(tokenized_datasets["train"]["labels"])

{0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22}

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from transformers import TrainingArguments
from transformers import Trainer

training_args = TrainingArguments("distilbert-cased-reviews-v1",
                                  hub_model_id="distilbert-cased-reviews-v1",
                                  evaluation_strategy="epoch",
                                  # fp16=True,
                                  # num_train_epochs=1,
                                  save_strategy="epoch",
                                  # learning_rate=2e-5,  =0.00005,
                                  # weight_decay=0.0,
                                  # lr_scheduler_type: SchedulerType | str = "linear",
                                  # weight_decay=0.01,
                                  push_to_hub=True,
                                  # per_device_train_batch_size=8,
                                  )
# training_args.set_dataloader(train_batch_size=16, eval_batch_size=32)

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=23) # ignore_mismatched_sizes=True

model.safetensors:   0%|          | 0.00/263M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'classifier.bias', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import evaluate
evaluate.list_evaluation_modules()

['lvwerra/test',
 'jordyvl/ece',
 'angelina-wang/directional_bias_amplification',
 'cpllab/syntaxgym',
 'lvwerra/bary_score',
 'hack/test_metric',
 'yzha/ctc_eval',
 'codeparrot/apps_metric',
 'mfumanelli/geometric_mean',
 'daiyizheng/valid',
 'erntkn/dice_coefficient',
 'mgfrantz/roc_auc_macro',
 'Vlasta/pr_auc',
 'gorkaartola/metric_for_tp_fp_samples',
 'idsedykh/metric',
 'idsedykh/codebleu2',
 'idsedykh/codebleu',
 'idsedykh/megaglue',
 'cakiki/ndcg',
 'Vertaix/vendiscore',
 'GMFTBY/dailydialogevaluate',
 'GMFTBY/dailydialog_evaluate',
 'jzm-mailchimp/joshs_second_test_metric',
 'ola13/precision_at_k',
 'yulong-me/yl_metric',
 'abidlabs/mean_iou',
 'abidlabs/mean_iou2',
 'KevinSpaghetti/accuracyk',
 'NimaBoscarino/weat',
 'ronaldahmed/nwentfaithfulness',
 'Viona/infolm',
 'kyokote/my_metric2',
 'kashif/mape',
 'Ochiroo/rouge_mn',
 'giulio98/code_eval_outputs',
 'leslyarun/fbeta_score',
 'giulio98/codebleu',
 'anz2/iliauniiccocrevaluation',
 'zbeloki/m2',
 'xu1998hz/sescore',
 'dvit

In [ ]:
import evaluate
import numpy as np
def compute_metrics(eval_preds):
    # metric = evaluate.load("yyu/amazon-attrprompt")
    # acc = evaluate.load("accuracy")
    # f1 = evaluate.load("f1")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    acc_metric = evaluate.load("accuracy").compute(predictions=predictions, references=labels)
    f1_metric = evaluate.load("f1").compute(predictions=predictions, references=labels, average='weighted')
    return {'accuracy': acc_metric, 'f1_score': f1_metric}

In [ ]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1 Score
1,0.251400,1.842164,{'accuracy': 0.7347826086956522},{'f1': 0.7217427746565059}
2,0.270900,1.875472,{'accuracy': 0.7347826086956522},{'f1': 0.7200496580345588}
3,0.091200,1.902153,{'accuracy': 0.7478260869565218},{'f1': 0.7350319489971969}


Trainer is attempting to log a value of "{'accuracy': 0.7347826086956522}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.7217427746565059}" of type <class 'dict'> for key "eval/f1_score" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.7347826086956522}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.7200496580345588}" of type <class 'dict'> for key "eval/f1_score" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.7478260869565218}" of type <clas

TrainOutput(global_step=5184, training_loss=0.2028955493444278, metrics={'train_runtime': 616.7992, 'train_samples_per_second': 67.218, 'train_steps_per_second': 8.405, 'total_flos': 3741341309525064.0, 'train_loss': 0.2028955493444278, 'epoch': 3.0})

In [ ]:
trainer.push_to_hub("End of training")

In [ ]:
label_names = ["magazines",
"camera_photo",
"office_products",
"kitchen",
"cell_phones_service",
"computer_video_games",
"grocery_and_gourmet_food",
"tools_hardware",
"automotive",
"music_album",
"health_and_personal_care",
"electronics",
"outdoor_living",
"video",
"apparel",
"toys_games",
"sports_outdoors",
"books",
"software",
"baby",
"musical_and_instruments",
"beauty",
"jewelry_and_watches"]

In [ ]:
model.config.id2label = {str(i): lb for i, lb in enumerate(label_names)}
model.config.label2id = {lb: str(i) for i, lb in enumerate(label_names)}

In [ ]:
model.config.push_to_hub("distilbert-cased-reviews-v1")

CommitInfo(commit_url='https://huggingface.co/Asteriks/distilbert-cased-reviews-v1/commit/b6a0eead8b32643998e80cb09cad8c7af97df0c9', commit_message='Upload config', commit_description='', oid='b6a0eead8b32643998e80cb09cad8c7af97df0c9', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
###

In [ ]:
tokenized_datasets["test"]["text"][100]

"Since the Phileas came out a number of years ago, I've had 4 of them (3 fountain and 1 roller ball). These fountain pens have never let me down and have always been sound in quality. The first was the original red and blue series, the others were later blue and green marble models. They look nice and write very well. Many fountain pen users regard the Phileas series as the best place to start with fountain pens, and I agree with that. They're easy to clean, easy to use, and mine have always been ready to write for me. I whole heartedly suggest this pen as a fine addition to any pen collection, as well as for a nice fountain pen to use on a daily basis"

In [ ]:
predictions_val = trainer.predict(tokenized_datasets["test"])
print(predictions_val.predictions.shape, predictions_val.label_ids.shape)

(1130, 23) (1130,)


In [ ]:
import numpy as np
preds_val = np.argmax(predictions_val.predictions, axis=-1)

In [ ]:
import evaluate
acc = evaluate.load("accuracy")
f1 = evaluate.load("f1")
print(acc.compute(predictions=preds_val, references=predictions_val.label_ids))
print(f1.compute(predictions=preds_val, references=predictions_val.label_ids, average='weighted'))

{'accuracy': 0.815929203539823}
{'f1': 0.805333877201633}


In [ ]:
from transformers import pipeline

pipe = pipeline("text-classification", model="Asteriks/distilbert-cased-reviews-v1")

config.json:   0%|          | 0.00/1.88k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/263M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/669k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
output = pipe("Red Hot Chili Peppers on vinyl has been a disappointing experience.. I had to return both “By The Way” and “Stadium Arcadium” because there were skips on almost all of it.. Kind of made it seem like the record label just went cheap, which is a disservice to anyone that actually listens to their vinyl... This “Greatest Hits” compilation did not have the same problems as the other two I bought. It sounded as it should have, and there were no skips.")

In [ ]:
output

[{'label': 'music_album', 'score': 0.9999582767486572}]